In [2]:
import os
import csv
import numpy as np
from glob import glob
import pandas as pd
from tqdm import tqdm
import json
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import argparse
import os
import random

import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle


/Users/xuhuimin/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/xuhuimin/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/xuhuimin/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/xuhuimin/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/xuhuimin/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/xuhuimin/anaconda3/lib/

### datasets

In [2]:
seed = 3535999445

def _rocstories(path):
    with open(path, encoding='utf_8') as f:
        f = csv.reader(f)
        st = []
        y = []
        for i, line in enumerate(tqdm(list(f), ncols=80, leave=False)):
            if i > 0:
                s = ' '.join(line[1:5])
                st.append(s)
                y.append(line[4+int(line[-1])])
        return st, y

In [3]:
def rocstories(data_dir, n_train=1497, n_valid=374):
    storys, ys = _rocstories(os.path.join(data_dir, 'cloze_test_val__spring2016 - cloze_test_ALL_val.csv'))
    teX1,  _ = _rocstories(os.path.join(data_dir, 'cloze_test_test__spring2016 - cloze_test_ALL_test.csv'))
    tr_storys, va_storys, tr_ys, va_ys = train_test_split(storys, ys, test_size=n_valid, random_state=seed)
    trX1 = []
    trY = []
    for s, y in zip(tr_storys, tr_ys):
        trX1.append(s)
        trY.append(y)

    vaX1 = []
    vaY = []
    for s, y in zip(va_storys, va_ys):
        vaX1.append(s)
        vaY.append(y)
    trY = np.asarray(trY)
    vaY = np.asarray(vaY)
    return (trX1, trY), (vaX1, vaY), (teX1)

In [4]:
(trX1, trY), (vaX1, vaY), (teX1)=rocstories('./ROCStory/', n_train=1497, n_valid=374)

FileNotFoundError: [Errno 2] No such file or directory: './ROCStory/cloze_test_val__spring2016 - cloze_test_ALL_val.csv'

In [5]:
seed = 3535999445

def _rocstories(path):
    with open(path, encoding='utf_8') as f:
        f = csv.reader(f)
        st = []
        ct1 = []
        ct2 = []
        y = []
        for i, line in enumerate(tqdm(list(f), ncols=80, leave=False)):
            if i > 0:
                s = ' '.join(line[1:5])
                c1 = line[5]
                c2 = line[6]
                st.append(s)
                ct1.append(c1)
                ct2.append(c2)
                y.append(int(line[-1])-1)
        return st, ct1, ct2, y


In [6]:
def rocstories(data_dir, n_train=1497, n_valid=374):
    storys, comps1, comps2, ys = _rocstories(os.path.join(data_dir, 'cloze_test_val__spring2016 - cloze_test_ALL_val.csv'))
    teX1, teX2, teX3, _ = _rocstories(os.path.join(data_dir, 'cloze_test_test__spring2016 - cloze_test_ALL_test.csv'))
    tr_storys, va_storys, tr_comps1, va_comps1, tr_comps2, va_comps2, tr_ys, va_ys = train_test_split(storys, comps1, comps2, ys, test_size=n_valid, random_state=seed)
    trX1, trX2, trX3 = [], [], []
    trY = []
    for s, c1, c2, y in zip(tr_storys, tr_comps1, tr_comps2, tr_ys):
        trX1.append(s)
        trX2.append(c1)
        trX3.append(c2)
        trY.append(y)

    vaX1, vaX2, vaX3 = [], [], []
    vaY = []
    for s, c1, c2, y in zip(va_storys, va_comps1, va_comps2, va_ys):
        vaX1.append(s)
        vaX2.append(c1)
        vaX3.append(c2)
        vaY.append(y)
    trY = np.asarray(trY, dtype=np.int32)
    vaY = np.asarray(vaY, dtype=np.int32)
    return (trX1, trX2, trX3, trY), (vaX1, vaX2, vaX3, vaY), (teX1, teX2, teX3)

In [49]:
def children_book(data_dir):
    with open(data_dir) as f:
        data=f.readlines()

    data=[data[22*i:22*(i+1)] for i in range(int(len(data)/22))]

    X,Y=[],[]
    for d in range(len(data)):
        x=[]
        for i in data[d][:20]:
            x.append(' '.join(i.split(' ')[1:]).split('\n')[0])
        X.append(x)
        y=' '.join(data[d][20].split(' ')[1:])
        y=y.replace('XXXXX',y.split('\t')[1]).split('\t')[0]
        Y.append(y)
    return X,Y

In [50]:
teX1,teY1=children_book('./CBTest/data/cbtest_CN_test_2500ex.txt')

In [51]:
teX2,teY2=children_book('./CBTest/data/cbtest_NE_test_2500ex.txt')

In [52]:
teX=teX1+teX2
teY=teY1+teY2

In [53]:
df=pd.DataFrame()
df['InputStoryid']=0
df['InputSentence1']=teX
df['InputSentence2']=0
df['InputSentence3']=0
df['InputSentence4']=0
df['RandomFifthSentenceQuiz1']=teY
df['RandomFifthSentenceQuiz2']=0
df['AnswerRightEnding']=0

In [68]:
df['InputSentence1']=df['InputSentence1'].apply(lambda x: "".join(x))

In [69]:
len(df)

5000

In [75]:
df.to_csv('./CBTest/CBTest_5000.csv',index=False)

In [76]:
trX1,trY1=children_book('./CBTest/data/cbtest_CN_train.txt')
trX2,trY2=children_book('./CBTest/data/cbtest_NE_train.txt')

In [77]:
vaX1,vaY1=children_book('./CBTest/data/cbtest_CN_valid_2000ex.txt')
vaX2,vaY2=children_book('./CBTest/data/cbtest_NE_valid_2000ex.txt')

In [78]:
X=trX1+trX2+vaX1+vaX2
Y=trY1+trY2+vaY1+vaY2

In [79]:
df=pd.DataFrame()
df['InputStoryid']=0
df['InputSentence1']=X
df['InputSentence2']=0
df['InputSentence3']=0
df['InputSentence4']=0
df['RandomFifthSentenceQuiz1']=Y
df['RandomFifthSentenceQuiz2']=0
df['AnswerRightEnding']=0

In [80]:
df['InputSentence1']=df['InputSentence1'].apply(lambda x: "".join(x))

In [81]:
df.to_csv('./CBTest/CBTest_233488.csv',index=False)

In [361]:
df['InputSentence1']=df['InputSentence1'].apply(lambda x: x[0])

In [444]:
def race(data_dir):
    ide=[]
    article=[]
    questions=[]
    answers=[]
    files=glob(data_dir)
    for file in files:      
        with open(file) as f:
            data=f.read()

        data = json.loads(data)
        a=data["answers"]
        b=data['options']
        c=data['questions']
        d=data['article']
        e=data['id']
        diction={'A':0,'B':1,'C':2,'D':3,'E':4,'F':5}
        a=[diction[x] if x in diction else x for x in a]

        answer=[]
        for i in range(len(a)):
            answer.append(b[i][a[i]])
        ide.append(e)
        article.append(d)
        questions.append(c)
        answers.append(answer)
        
    return ide,article,questions,answers

In [445]:
teX1,teX2,teX3,teY1=race('./RACE/test/high/*.txt')
tex1,tex2,tex3,tey1=race('./RACE/test/middle/*.txt')

In [446]:
X1=teX1+tex1
X2=teX2+tex2
X3=teX3+tex3
Y1=teY1+tey1

In [447]:
df=pd.DataFrame()
df['InputStoryid']=X1
df['InputSentence1']=X2
df['InputSentence2']=X3
df['InputSentence3']=0
df['InputSentence4']=0
df['RandomFifthSentenceQuiz1']=Y1
df['RandomFifthSentenceQuiz2']=0
df['AnswerRightEnding']=0

In [452]:
df.to_csv('./RACE/RACE_1047.csv',index=False)

In [458]:
teX1,teX2,teX3,teY1=race('./RACE/dev/high/*.txt')
tex1,tex2,tex3,tey1=race('./RACE/dev/middle/*.txt')
teX10,teX20,teX30,teY10=race('./RACE/train/high/*.txt')
tex10,tex20,tex30,tey10=race('./RACE/train/middle/*.txt')

In [459]:
X1=teX1+tex1+teX10+tex10
X2=teX2+tex2+teX20+tex20
X3=teX3+tex3+teX30+tex30
Y1=teY1+tey1+teY10+tey10

In [460]:
df=pd.DataFrame()
df['InputStoryid']=X1
df['InputSentence1']=X2
df['InputSentence2']=X3
df['InputSentence3']=0
df['InputSentence4']=0
df['RandomFifthSentenceQuiz1']=Y1
df['RandomFifthSentenceQuiz2']=0
df['AnswerRightEnding']=0

In [461]:
len(df)

26526

In [462]:
df.to_csv('./RACE/RACE_26526.csv',index=False)

In [463]:
df.head()

,InputStoryid,InputSentence1,InputSentence2,InputSentence3,InputSentence4,RandomFifthSentenceQuiz1,RandomFifthSentenceQuiz2,AnswerRightEnding
0,high1735.txt,"It's cool, and it's hot, and everyone is doing...","[A blogger is a person _ ., According to th...",0,0,"[who writes diaries online, write down what th...",0,0
1,high12636.txt,A At a recent visit to the beautiful Keukenho...,"[By using metaphor , the author means that the...",0,0,[the first results we produce in a certain are...,0,0
2,high19763.txt,If you win the $1.5 billion Powerball jackpot ...,[What does Don McNay think of winning a lotter...,0,0,"[It doesn't necessarily bring fortune., Suppor...",0,0
3,high1090.txt,Summer in Europe is very nice for us to visit ...,[Which of the following statements is NOT ment...,0,0,"[Berlin has many parks and bridges., all the a...",0,0
4,high20135.txt,"As teachers, most of us have been taught that ...","[According to what Guskey said, the culture of...",0,0,"[students not take tests seriously at all, tes...",0,0


### train

In [5]:
seed = 3535999445

def _rocstories(path):
    with open(path, encoding='utf_8') as f:
        f = csv.reader(f)
        st = []
        ct1 = []
        ct2 = []
        y = []
        for i, line in enumerate(tqdm(list(f), ncols=80, leave=False)):
            if i > 0:
                s = ' '.join(line[1:5])
                c1 = line[5]
                c2 = line[6]
                st.append(s)
                ct1.append(c1)
                ct2.append(c2)
                y.append(int(line[-1])-1)
        return st, ct1, ct2, y

def rocstories(data_dir, n_train=1497, n_valid=374):
    storys, comps1, comps2, ys = _rocstories(os.path.join(data_dir, 'cloze_test_val__spring2016 - cloze_test_ALL_val.csv'))
    teX1, teX2, teX3, _ = _rocstories(os.path.join(data_dir, 'cloze_test_test__spring2016 - cloze_test_ALL_test.csv'))
    tr_storys, va_storys, tr_comps1, va_comps1, tr_comps2, va_comps2, tr_ys, va_ys = train_test_split(storys, comps1, comps2, ys, test_size=n_valid, random_state=seed)
    trX1, trX2, trX3 = [], [], []
    trY = []
    for s, c1, c2, y in zip(tr_storys, tr_comps1, tr_comps2, tr_ys):
        trX1.append(s)
        trX2.append(c1)
        trX3.append(c2)
        trY.append(y)

    vaX1, vaX2, vaX3 = [], [], []
    vaY = []
    for s, c1, c2, y in zip(va_storys, va_comps1, va_comps2, va_ys):
        vaX1.append(s)
        vaX2.append(c1)
        vaX3.append(c2)
        vaY.append(y)
    trY = np.asarray(trY, dtype=np.int32)
    vaY = np.asarray(vaY, dtype=np.int32)
    return (trX1, trX2, trX3, trY), (vaX1, vaX2, vaX3, vaY), (teX1, teX2, teX3)


In [7]:
(trX1, trX2, trX3, trY), (vaX1, vaX2, vaX3, vaY), (teX1, teX2, teX3)=rocstories('./generative_corpus/ROCStory/', n_train=1497, n_valid=374)

In [15]:
trY

array([0, 1, 1, ..., 1, 0, 1], dtype=int32)

In [16]:
class TextEncoder(object):
    """
    mostly a wrapper for a public python bpe tokenizer
    """

    def __init__(self, encoder_path, bpe_path):
        self.nlp = spacy.load('en', disable=['parser', 'tagger', 'ner', 'textcat'])
        self.encoder = json.load(open(encoder_path))
        self.decoder = {v:k for k,v in self.encoder.items()}
        merges = open(bpe_path, encoding='utf-8').read().split('\n')[1:-1]
        merges = [tuple(merge.split()) for merge in merges]
        self.bpe_ranks = dict(zip(merges, range(len(merges))))
        self.cache = {}

    def bpe(self, token):
        word = tuple(token[:-1]) + ( token[-1] + '</w>',)
        if token in self.cache:
            return self.cache[token]
        pairs = get_pairs(word)

        if not pairs:
            return token+'</w>'

        while True:
            bigram = min(pairs, key = lambda pair: self.bpe_ranks.get(pair, float('inf')))
            if bigram not in self.bpe_ranks:
                break
            first, second = bigram
            new_word = []
            i = 0
            while i < len(word):
                try:
                    j = word.index(first, i)
                    new_word.extend(word[i:j])
                    i = j
                except:
                    new_word.extend(word[i:])
                    break

                if word[i] == first and i < len(word)-1 and word[i+1] == second:
                    new_word.append(first+second)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            new_word = tuple(new_word)
            word = new_word
            if len(word) == 1:
                break
            else:
                pairs = get_pairs(word)
        word = ' '.join(word)
        if word == '\n  </w>':
            word = '\n</w>'
        self.cache[token] = word
        return word

    def encode(self, texts, verbose=True):
        texts_tokens = []
        if verbose:
            for text in tqdm(texts, ncols=80, leave=False):
                text = self.nlp(text_standardize(ftfy.fix_text(text)))
                text_tokens = []
                for token in text:
                    text_tokens.extend([self.encoder.get(t, 0) for t in self.bpe(token.text.lower()).split(' ')])
                texts_tokens.append(text_tokens)
        else:
            for text in texts:
                text = self.nlp(text_standardize(ftfy.fix_text(text)))
                text_tokens = []
                for token in text:
                    text_tokens.extend([self.encoder.get(t, 0) for t in self.bpe(token.text.lower()).split(' ')])
                texts_tokens.append(text_tokens)
        return texts_tokens

In [8]:
def transform_roc(X1, X2, X3):
    n_batch = len(X1)
    xmb = np.zeros((n_batch, 2, n_ctx, 2), dtype=np.int32)
    mmb = np.zeros((n_batch, 2, n_ctx), dtype=np.float32)
    start = encoder['_start_']
    delimiter = encoder['_delimiter_']
    for i, (x1, x2, x3), in enumerate(zip(X1, X2, X3)):
        x12 = [start] + x1[:max_len] + [delimiter] + x2[:max_len] + [clf_token]
        x13 = [start] + x1[:max_len] + [delimiter] + x3[:max_len] + [clf_token]
        l12 = len(x12)
        l13 = len(x13)
        xmb[i, 0, :l12, 0] = x12
        xmb[i, 1, :l13, 0] = x13
        mmb[i, 0, :l12] = 1
        mmb[i, 1, :l13] = 1
    # Position information that is added to the input embeddings in the TransformerModel
    xmb[:, :, :, 1] = np.arange(n_vocab + n_special, n_vocab + n_special + n_ctx)
    return xmb, mmb

In [15]:
summary=pd.read_csv('/Users/xuhuimin/Desktop/summary.csv')

In [16]:
No=summary[summary['shuffle']=='[1 2 3 4]'][['shuffle','story1','story2','0','1','max','pred']]

In [19]:
No[No['max']<=0.6]

,shuffle,story1,story2,0,1,max,pred
151,[1 2 3 4],We had dinner in Brooklyn tonight. The include...,We had dinner in Brooklyn tonight. The include...,0.516358,0.483642,0.516358,0
343,[1 2 3 4],Late last night I awoke to a strange sound of ...,Late last night I awoke to a strange sound of ...,0.457357,0.542643,0.542643,1
474,[1 2 3 4],Neil went to a celebration with his Vietnamese...,Neil went to a celebration with his Vietnamese...,0.542947,0.457053,0.542947,0
754,[1 2 3 4],Jim was using a selfie stick. He was sticking ...,Jim was using a selfie stick. He was sticking ...,0.500000,0.500000,0.500000,0
831,[1 2 3 4],My sister in law lost her husband in 1984. The...,My sister in law lost her husband in 1984. The...,0.536753,0.463247,0.536753,0
2356,[1 2 3 4],It had been raining for days. The sky had stay...,It had been raining for days. The sky had stay...,0.500000,0.500000,0.500000,0
2516,[1 2 3 4],It was a busy Saturday night at the airport as...,It was a busy Saturday night at the airport as...,0.410673,0.589327,0.589327,1
2684,[1 2 3 4],Maurice is preparing to celebrate Thanksgiving...,Maurice is preparing to celebrate Thanksgiving...,0.500000,0.500000,0.500000,0
2741,[1 2 3 4],Bridget as always sleepy. Bridget tried going ...,Bridget as always sleepy. Bridget tried going ...,0.500000,0.500000,0.500000,0
3004,[1 2 3 4],Our building lost the Internet yesterday. I ca...,Our building lost the Internet yesterday. I ca...,0.500000,0.500000,0.500000,0


In [18]:
No[No['max']>0.6]

,shuffle,story1,story2,0,1,max,pred
41,[1 2 3 4],Susan was excited to plan her first egg hunt i...,Susan was excited to plan her first egg hunt i...,0.989628,0.010372,0.989628,0
228,[1 2 3 4],Josh wanted nothing more than world peace. He ...,Josh wanted nothing more than world peace. He ...,0.000278,0.999722,0.999722,1
243,[1 2 3 4],Fred had a basketball game tonight. He was rat...,Fred had a basketball game tonight. He was rat...,0.999998,0.000002,0.999998,0
253,[1 2 3 4],Maurice watched the clown bump his way through...,Maurice watched the clown bump his way through...,0.240530,0.759470,0.759470,1
255,[1 2 3 4],Lindsay was very interested in guitar lessons....,Lindsay was very interested in guitar lessons....,0.167398,0.832602,0.832602,1
260,[1 2 3 4],I was walking to my office complex. I noticed ...,I was walking to my office complex. I noticed ...,0.364761,0.635239,0.635239,1
285,[1 2 3 4],"My cousin decided to pick me up, and take me t...","My cousin decided to pick me up, and take me t...",0.004362,0.995638,0.995638,1
372,[1 2 3 4],Jennifer needed to travel for work. She bought...,Jennifer needed to travel for work. She bought...,0.029068,0.970932,0.970932,1
410,[1 2 3 4],Tim has never cooked for his family. He was ne...,Tim has never cooked for his family. He was ne...,0.624466,0.375534,0.624466,0
428,[1 2 3 4],Roger wanted to play tennis as well as Roger F...,Roger wanted to play tennis as well as Roger F...,0.688027,0.311973,0.688027,0


### spacy

In [1]:
import spacy

/Users/xuhuimin/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/xuhuimin/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [2]:
nlp=spacy.load('en')

/Users/xuhuimin/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/xuhuimin/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [72]:
document=["Cinderella was a lovely girl. ",
"Her old mother died. ",
"Her father married a wife, ", 
"and the stepmother has brought two elder sisters, ",
"She does a lot of housework everyday. ",
"So the girl's clothes were very dirty. ",
"Everybody called her Cinderella. ",
"The prince gave all the females an invitation, ", 
"asked them to come in the palace to join the dance party. ", 
"But Cinderella’s stepmother didn’t let her go. ",
"The day of the party arrived. Cinderella met a fairy godmother. ",
"She gave Cinderella a coach, two shoots, two horses and a beautiful dress. ",
"But magic will stop working at midnight. ",
"Cinderella arrived at the palace and walked into the dance hall. ", 
"The Prince saw her. He danced only with Cinderella, and he fell in love with her. ",
"A clock chiming reminded Cinderella of her fairy godmother's warning. ",
"She ran out of the palace. The Prince only found Cinderella’s one glass shoot. ",
"The next morning, The Prince proclaimed: Whoever the shoot fits, shall be wife to him. ",
"He arrived at the stepmother's house. ",
"And Cinderella’s sisters can’t fit the shoot. But she can. ",
"Then Cinderella took the other shoot from her pocket and put it on. ",
"Finally, the Prince and Cinderella were married. They lived happily ever after. "]

In [73]:
document=''.join(document)

In [74]:
document=nlp(document)

In [75]:
type(document)

spacy.tokens.doc.Doc

In [76]:
len(document)

247

In [77]:
document[1].pos_

'VERB'

In [78]:
all_tags={w.pos:w.pos_ for w in document}

In [79]:
all_tags

{83: 'ADJ',
 84: 'ADP',
 85: 'ADV',
 88: 'CCONJ',
 89: 'DET',
 91: 'NOUN',
 92: 'NUM',
 93: 'PART',
 94: 'PRON',
 95: 'PROPN',
 96: 'PUNCT',
 99: 'VERB'}

In [80]:
for word in list(document.sents)[4]:
    print(word,word.tag_)

Everybody NN
called VBD
her PRP$
Cinderella NNP
. .


In [81]:
#define some parameters  
noisy_pos_tags = ["SPACE"]
min_token_length = 2
 
#Function to check if the token is a noise or not  
def isNoise(token):     
    is_noise = False
    if token.pos_ in noisy_pos_tags:
        is_noise = True 
    elif token.is_stop == True:
        is_noise = True
    elif len(token.string) <= min_token_length:
        is_noise = True
    return is_noise 
def cleanup(token, lower = True):
    if lower:
        token = token.lower() 
    return token.strip()
from collections import Counter
cleaned_list = [cleanup(word.string) for word in document if not isNoise(word)]


In [82]:
Counter(cleaned_list).most_common(2)

[('cinderella', 12), ('the', 6)]

In [83]:
labels = set([w.label_ for w in document.ents]) 

In [84]:
labels

{'CARDINAL', 'DATE', 'FAC', 'PERSON', 'TIME'}

In [85]:
for label in labels: 
    entities = [cleanup(e.string, lower=False) for e in document.ents if label==e.label_] 
    entities = list(set(entities)) 
    print (label,entities)

DATE ['The day']
TIME ['midnight', 'The next morning']
PERSON ['Cinderella']
FAC ['Cinderella']
CARDINAL ['two', 'one']


In [102]:
def pos_words (document, token, pos_tag):
    sentences = [sent for sent in document.sents if token in sent.string]     
    pwrds = []
    for sent in sentences:
        for word in sent:
            if token in word.string: 
                   pwrds.extend([child.string.strip() for child in word.children
                                                      if child.pos_ == pos_tag] )
    return Counter(pwrds).most_common(10)

pos_words(document, 'Cinderella', "VERB")

[]